* [Titanic 100%](https://www.kaggle.com/blurredmachine/titanic-survival-a-complete-guide-for-beginners)
* [MLCource](https://mlcourse.ai/)

In [ ]:
import os
import warnings
import seaborn as sns
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

sns.set()

warnings.filterwarnings("ignore")


df = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')

In [ ]:
# describe(include = ['O'])* will show the descriptive statistics of object data types.
df.describe(include=['O'])

In [ ]:
df.info()

In [ ]:
survived = df[df['Survived'] == 1]
not_survived = df[df['Survived'] == 0]

print('Survived: %i (%.1f%%)' %
      (len(survived), float(len(survived))/len(df)*100))
print('Not Survived: %i (%.1f%%)' %
      (len(not_survived), float(len(not_survived))/len(df)*100))
print('Total: %i' % len(df))

# Гистограмма сравнения двух типов откликов по одному предиктору, с ликвидацией NaN
plt.figure(figsize=[15, 5])
plt.subplot(111)
sns.distplot(survived['Age'].dropna().values,
             bins=range(0, 81, 1), kde=True, color='blue')
sns.distplot(not_survived['Age'].dropna().values, bins=range(
    0, 81, 1), kde=True, color='red', axlabel='Age')

In [ ]:
df.Pclass.value_counts()

In [ ]:
# Вектор показывает группировку внутри колоники по стороннему параметру
pclass_survived = df.groupby('Pclass').Survived.value_counts()
pclass_survived

In [ ]:
# Метод анстак позволяет разделить отклик на группы по значению предикторов
pclass_survived.unstack(level=0).plot(kind='bar', subplots=False)

In [ ]:
# Фрейм группирует предикторы по значению отклика
pclass_survived_average = df[['Pclass', 'Survived']].groupby(['Pclass'],
                                                             as_index=False).mean()
pclass_survived_average

In [ ]:
# График группировки по двум параметрам
sns.barplot(x='Pclass', y='Survived', data=df)

In [ ]:
# Создание фрейма с кроссраспределением групп по двум предикторам и их итоговому отклику
tab=pd.crosstab(df['Pclass'], df['Sex'])
print(tab)

# График, который через суммирование всех значений в колонке и превращение в дробь, показывает процентное распределение в ней
tab.div(tab.sum(1). astype(float), axis=0).plot(kind='bar', stacked=False)
plt.xlabel('Pclass')
plt.ylabel('Percentage')

In [ ]:
# График сравнения откликов в группах, в зависимости от предиктора (hue) 
sns.factorplot('Sex', 'Survived', hue='Pclass', size=4, aspect=2, data=df)

In [ ]:
# Зависимость и распределение откликов от трех предикторов. Для этого строятся параллельные графики по два в каждом
sns.factorplot(x='Pclass', y='Survived', hue='Sex', col='Embarked', data=df);

In [ ]:
# Относительное сравнение. Один предиктор постоянен, отклик отправлен в (hue), дополнительные предикторы меняются
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

sns.violinplot(x='Embarked', y='Age', hue='Survived',
               data=df, split=True, ax=ax1)
sns.violinplot(x='Pclass', y='Age', hue='Survived',
               data=df, split=True, ax=ax2)
sns.violinplot(x='Sex', y='Age', hue='Survived', data=df, split=True, ax=ax3)

In [ ]:
# Построение графика параллельного сравнения двух откликов для одного предиктора для двух групп

male_survived = df[(df['Survived'] == 1) & (df['Sex'] == 'male')]
female_survived = df[(df['Survived'] == 1) & (df['Sex'] == 'female')]
male_not_survived = df[(df['Survived'] == 0) & (df['Sex'] == 'male')]
female_not_survived = df[(df['Survived'] == 0) & (df['Sex'] == 'female')]

plt.figure(figsize=[15, 5])
plt.subplot(121)
sns.distplot(female_survived['Age'].dropna().values,
             bins=range(0, 81, 1), kde=True, color='blue')
sns.distplot(female_not_survived['Age'].dropna().values, bins=range(
    0, 81, 1), kde=True, color='red', axlabel='Female Age')

plt.subplot(122)
sns.distplot(male_survived['Age'].dropna().values,
             bins=range(0, 81, 1), kde=True, color='blue')
sns.distplot(male_not_survived['Age'].dropna().values, bins=range(
    0, 81, 1), kde=True, color='red', axlabel='Male Age')

In [ ]:
# Хитмап корреляции для числовых колонок
plt.figure(figsize=(15, 6))
sns.heatmap(df.drop('PassengerId', axis=1).corr(),
            vmax=0.6, square=True, annot=True)

In [ ]:
# Создаем список, что позволит работать циклами со всеми вложенными в него фреймами 
trd = [df, tsd]

In [ ]:
for dataset in trd:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.') #Извлекает все буквы перед избранным знаком(.)

In [ ]:
df.head()

In [ ]:
pd.crosstab(df['Title'], df['Sex'])

In [ ]:
# Объединяем группы, сливая единичные параметры в более общие
for d in trd:
    d['Title'] = d['Title'].replace(
        ['Capt', 'Col', 'Countess', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Rev', 'Sir'], 'Other')
    d['Title'] = d['Title'].replace('Mlle', 'Miss')
    d['Title'] = d['Title'].replace('Ms', 'Miss')
    d['Title'] = d['Title'].replace('Mme', 'Mrs')

In [ ]:
title_mapping = {'Mr': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Other': 5}
for d in trd:
    d['Title'] = d['Title'].map(title_mapping)
    d['Title'] = d['Title'].fillna(0)

In [ ]:
for d in trd:
    d['Sex'] = d['Sex'].map({'female':0, 'male':1}).astype(int)

In [ ]:
for d in trd:
    d['Embarked'] = d['Embarked'].fillna('S')

In [ ]:
# Все пустые строки заполняем случайным числом из охвата +-сигма
for d in trd:
    age_avg = d['Age'].mean()
    age_std = d['Age'].std()
    age_null_count = d['Age'].isnull().sum()

    age_null_random_list = np.random.randint(
        age_avg - age_std, age_avg + age_std, size=age_null_count)
    d['Age'][np.isnan(d['Age'])] = age_null_random_list
    d['Age'] = d['Age'].astype(int)

In [ ]:
# Разделяем данные на равные блоки методом CUT
df['AgeBand'] = pd.cut(df['Age'], 5) #Указываем число нужных блоков
print(df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean())

In [ ]:
# В списке заменяем исходные параметры по заданному значению (если предварительно определена группа)
for d in trd:
    d.loc[d['Age'] <= 16, 'Age'] = 0
    d.loc[(d['Age'] > 16) & (d['Age'] <= 32), 'Age'] = 1
    d.loc[(d['Age'] > 32) & (d['Age'] <= 48), 'Age'] = 2
    d.loc[(d['Age'] > 48) & (d['Age'] <= 64), 'Age'] = 3
    d.loc[d['Age'] > 64, 'Age'] = 4

In [ ]:
for d in trd:
    d['Embarked'] = d['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
sns.barplot(x='Title', y='Survived', ci=None, data=df) # ci убирает доп.черты на графике

In [ ]:
df['Title'] = df['Title'].map(
    {'Mr': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Other': 5})

In [ ]:
for d in trd:
    d['Fare'] = d['Fare'].fillna(df['Fare'].median())

In [ ]:
# Метод qcut делит колонку на квантилиб что делает число входов равным
df['FareBand'] = pd.qcut(df['Fare'], 4)
print(df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean())

In [ ]:
for d in trd:
    d.loc[d['Fare'] <= 7.91, 'Fare'] = 0
    d.loc[(d['Fare'] > 7.91) & (d['Fare'] <= 14.454), 'Fare'] = 1
    d.loc[(d['Fare'] > 14.454) & (d['Fare'] <= 31.0), 'Fare'] = 2
    d.loc[d['Fare'] > 31.0, 'Fare'] = 3

In [ ]:
tsd.head()

In [ ]:
for d in trd:
    d['FamilySize'] = d['SibSp'] + d['Parch'] + 1
    
print(df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

In [ ]:
sns.barplot(x='FamilySize', y='Survived', ci=False, data=df)

In [ ]:
for d in trd:
    d['IsAlone'] = 0
    d.loc[d['FamilySize'] == 1, 'IsAlone'] = 1

print(df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())